## Machinery

In [64]:
%load_ext watermark
%watermark -i -v -m -p pandas,pystan,arviz

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2021-01-10T21:37:05+08:00

CPython 3.8.6
IPython 7.19.0

pandas 1.2.0
pystan 2.19.1.1
arviz 0.10.0

compiler   : GCC 9.3.0
system     : Linux
release    : 5.8.0-36-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit


In [65]:
import pandas as pd
import numpy as np
from termcolor import colored

%load_ext rpy2.ipython

import epiweeks

import os
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='arviz')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [66]:
%matplotlib inline
# Make inline plots raster graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib.lines import Line2D

# Parameters for seaborn plots
import seaborn as sns
clrs = sns.color_palette("Spectral", 6)
def set_plot_style(usetex=False):
    sns.set_style('white', {'axes.linewidth': 0.5})
    sns.set(style='white', font_scale=1.1,#context='paper',
            rc={'xtick.major.size': 6, 'ytick.major.size': 6, 'legend.fontsize': 14,
                'text.usetex': usetex, 'font.family': 'serif', 'font.serif': ['Verdana'],
                'text.latex.preamble': r"\usepackage{type1cm}"}) 
    plt.rcParams['xtick.major.size'] = 6
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 6
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True
    
set_plot_style()

month_names = ["Unknown", "January", "Febuary", "March", "April", "May", "June", "July", 
               "August", "September", "October", "November", "December"]

%config InlineBackend.figure_format = 'retina'

In [67]:
import pystan
import arviz as az

func_dict = {"q2.5": lambda x: np.percentile(x, 2.5), 
             "q25": lambda x: np.percentile(x, 25), 
             "median": lambda x: np.percentile(x, 50), 
             "q75": lambda x: np.percentile(x, 75), 
             "q97.5": lambda x: np.percentile(x, 97.5)}

def get_stats(cmdstan_data, varnames):
    # include mean and hdi
    stats = az.summary(cmdstan_data, var_names=varnames, hdi_prob=0.95).loc[:, ['mean','hdi_2.5%','hdi_97.5%','ess_bulk','ess_tail','r_hat']].reset_index().rename(columns={'index':'var', 'hdi_2.5%':'hdi2.5', 'hdi_97.5%':'hdi97.5'})
    stats = az.summary(cmdstan_data, var_names=varnames, hdi_prob=0.50).loc[:, ['hdi_25%','hdi_75%']].reset_index().rename(columns={'index':'var', 'hdi_25%':'hdi25', 'hdi_75%':'hdi75'}).\
        merge(stats, left_on='var', right_on='var')
    # include percentiles
    stats = az.summary(cmdstan_data, var_names=varnames, stat_funcs=func_dict, extend=False).reset_index().rename(columns={'index': 'var'}).merge(stats, left_on='var', right_on='var')
    stats['time'] = stats['var'].apply(lambda st: st[st.find("[")+1:st.find("]")])
    stats['time'] = ['NA' if "[" not in y else int(x)+1 for x,y in zip(stats['time'],stats['var'])]
    stats['var'] = stats['var'].apply(lambda st: st[:st.find("[")] if "[" in st else st)
    return stats.loc[:,['var','time','mean','hdi2.5','hdi25','hdi75','hdi97.5','q2.5','q25','median','q75','q97.5','ess_bulk','ess_tail','r_hat']]

def get_stats_2d(cmdstan_data, varnames):
    # include mean and hpd
    stats = az.summary(cmdstan_data, var_names=varnames, hdi_prob=0.95).loc[:, ['mean','hdi_2.5%','hdi_97.5%','ess_bulk','ess_tail','r_hat']].reset_index().rename(columns={'index':'var', 'hdi_2.5%':'hdi2.5', 'hdi_97.5%':'hdi97.5'})
    stats = az.summary(cmdstan_data, var_names=varnames, hdi_prob=0.50).loc[:, ['hdi_25%','hdi_75%']].reset_index().rename(columns={'index':'var', 'hdi_25%':'hdi25', 'hdi_75%':'hdi75'}).\
        merge(stats, left_on='var', right_on='var')
    # include percentiles
    stats = az.summary(cmdstan_data, var_names=varnames, stat_funcs=func_dict, extend=False).reset_index().rename(columns={'index': 'var'}).merge(stats, left_on='var', right_on='var')
    stats['time'] = stats['var'].apply(lambda st: st[st.find("[")+1:st.find("]")])
    stats['time'] = ['NA' if "[" not in y else x for x,y in zip(stats['time'],stats['var'])]
    stats['var'] = stats['var'].apply(lambda st: st[:st.find("[")] if "[" in st else st)
    return stats.loc[:,['var','time','mean','hdi2.5','hdi25','hdi75','hdi97.5','q2.5','q25',
                        'median','q75','q97.5','ess_bulk','ess_tail','r_hat']]

In [68]:
!mkdir -p ../../figures
!mkdir -p ../../figures/draft

standistribdir = '../../../../CmdStan'
stanworkdir = '../../../NTU_Backup/mortality-US_all_states_20210107/historical'
!mkdir -p {stanworkdir}

In [69]:
datadir = "../../data_raw/mortality/cdc\ 2014-2018"
datafiles = !ls {datadir}/*csv
datafiles

['../../data_raw/mortality/cdc 2014-2018/Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2014-2018.csv']

In [70]:
df_ = pd.read_csv(datafiles[0])
df_[:5]

,Jurisdiction of Occurrence,MMWR Year,MMWR Week,Week Ending Date,All Cause,Natural Cause,Septicemia (A40-A41),Malignant neoplasms (C00-C97),Diabetes mellitus (E10-E14),Alzheimer disease (G30),...,flag_neopl,flag_diab,flag_alz,flag_inflpn,flag_clrd,flag_otherresp,flag_nephr,flag_otherunk,flag_hd,flag_stroke
0,Alabama,2014,1,01/04/2014,355,327,NaN,60,NaN,10,...,NaN,Suppressed (counts 1-9),NaN,Suppressed (counts 1-9),NaN,Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,NaN
1,Alabama,2014,2,01/11/2014,872,792,23,163,23,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2014,3,01/18/2014,"1,044",971,21,209,34,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,2014,4,01/25/2014,"1,022",967,25,205,23,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2014,5,02/01/2014,"1,040",953,18,200,26,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df = df_.rename(columns={'Jurisdiction of Occurrence':'jurisdiction', 
                    'MMWR Year':'year', 'MMWR Week':'week',
                    'All  Cause':'number_of_deaths'})\
    .loc[:,['jurisdiction','year','week','number_of_deaths']].reset_index(drop=True)
for x in ['year', 'week', 'number_of_deaths']:
    # small technical issue with "," in numbers for deaths
    if (type(df['number_of_deaths'][0])==str)&(x=='number_of_deaths'):
        df[x] = df[x].str.replace(",","")
    df[x] = df[x].astype('float').astype(pd.Int64Dtype())
df

,jurisdiction,year,week,number_of_deaths
0,Alabama,2014,1,355
1,Alabama,2014,2,872
2,Alabama,2014,3,1044
3,Alabama,2014,4,1022
4,Alabama,2014,5,1040
...,...,...,...,...
14089,United States,2018,48,55210
14090,United States,2018,49,56095
14091,United States,2018,50,56530
14092,United States,2018,51,56689


In [72]:
##adding 2019 
datadir = "../../data_raw/mortality/cdc"
datafiles = !ls {datadir}/*csv
datafiles

['../../data_raw/mortality/cdc/2020W34 (20200822; updated 20200902) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W35 (20200829; updated 20200909) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W40 (20201003; updated 20201015) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W41 (20201010; updated 20201022) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W42 (20201017; updated 20201029) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W43 (20201024; updated 20201103) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/2020W44 (20201031; updated 20201110) - Weekly_Counts_of_Deaths_by_State_and_Select_Causes__2019-2020.csv',
 '../../data_raw/mortality/cdc/202

In [73]:
df__ = pd.read_csv(datafiles[-1])
df__[:5]

,Jurisdiction of Occurrence,MMWR Year,MMWR Week,Week Ending Date,All Cause,Natural Cause,Septicemia (A40-A41),Malignant neoplasms (C00-C97),Diabetes mellitus (E10-E14),Alzheimer disease (G30),...,flag_alz,flag_inflpn,flag_clrd,flag_otherresp,flag_nephr,flag_otherunk,flag_hd,flag_stroke,flag_cov19mcod,flag_cov19ucod
0,Alabama,2019,1,2019-01-05,"1,077",993,30.0,198,22,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2019,2,2019-01-12,"1,090",994,25.0,187,24,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2019,3,2019-01-19,"1,114","1,042",22.0,238,18,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,2019,4,2019-01-26,"1,063",994,21.0,165,22,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2019,5,2019-02-02,"1,095","1,026",18.0,199,19,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df__.columns

Index(['Jurisdiction of Occurrence', 'MMWR Year', 'MMWR Week',
       'Week Ending Date', 'All Cause', 'Natural Cause',
       'Septicemia (A40-A41)', 'Malignant neoplasms (C00-C97)',
       'Diabetes mellitus (E10-E14)', 'Alzheimer disease (G30)',
       'Influenza and pneumonia (J09-J18)',
       'Chronic lower respiratory diseases (J40-J47)',
       'Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)',
       'Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)',
       'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)',
       'Diseases of heart (I00-I09,I11,I13,I20-I51)',
       'Cerebrovascular diseases (I60-I69)',
       'COVID-19 (U071, Multiple Cause of Death)',
       'COVID-19 (U071, Underlying Cause of Death)', 'flag_allcause',
       'flag_natcause', 'flag_sept', 'flag_neopl', 'flag_diab', 'flag_alz',
       'flag_inflpn', 'flag_clrd', 'flag_otherresp', 'flag_nephr',
       'flag_otherunk', 

In [75]:
df__ = df__.rename(columns={'Jurisdiction of Occurrence':'jurisdiction', 
                    'MMWR Year':'year', 'MMWR Week':'week',
                    'All Cause':'number_of_deaths'})\
    .loc[lambda d: (d['year']==2019) | ((d['year']==2020)&(d['week']<=8)), ['jurisdiction','year','week','number_of_deaths']].reset_index(drop=True)
for x in ['year', 'week', 'number_of_deaths']:
    # small technical issue with "," in numbers for deaths
    if (type(df__['number_of_deaths'][0])==str)&(x=='number_of_deaths'):
        df__[x] = df__[x].str.replace(",","")
    df__[x] = df__[x].astype('float').astype(pd.Int64Dtype())
df__

,jurisdiction,year,week,number_of_deaths
0,Alabama,2019,1,1077
1,Alabama,2019,2,1090
2,Alabama,2019,3,1114
3,Alabama,2019,4,1063
4,Alabama,2019,5,1095
...,...,...,...,...
3235,United States,2020,4,59137
3236,United States,2020,5,58795
3237,United States,2020,6,59367
3238,United States,2020,7,58782


In [76]:
df = df.append(df__)

In [77]:
# removing United States
df = df.loc[lambda d: d.jurisdiction!='United States']
df[:5]

,jurisdiction,year,week,number_of_deaths
0,Alabama,2014,1,355
1,Alabama,2014,2,872
2,Alabama,2014,3,1044
3,Alabama,2014,4,1022
4,Alabama,2014,5,1040


In [78]:
jurisdictions = df.jurisdiction.unique()
str(jurisdictions)

"['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'\n 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'\n 'Hawaii' 'Wyoming' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas'\n 'Kentucky' 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan'\n 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada'\n 'New Hampshire' 'New Jersey' 'New Mexico' 'New York' 'New York City'\n 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'\n 'Puerto Rico' 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee'\n 'Texas' 'Utah' 'Vermont' 'Virginia' 'Washington' 'West Virginia'\n 'Wisconsin']"

In [79]:
df_index_jurisdictions = pd.DataFrame({'jurisdiction': jurisdictions}).reset_index().rename(columns={'index': 'idx_jurisdiction'})
df_index_jurisdictions['idx_jurisdiction'] += 1
df_index_jurisdictions[:7]

,idx_jurisdiction,jurisdiction
0,1,Alabama
1,2,Alaska
2,3,Arizona
3,4,Arkansas
4,5,California
5,6,Colorado
6,7,Connecticut


In [81]:
years = list(df['year'].unique())
df_index_years = pd.DataFrame({'year': years}).reset_index().rename(columns={'index': 'idx_year'})
df_index_years['idx_year'] += 1
df_index_years

,idx_year,year
0,1,2014
1,2,2015
2,3,2016
3,4,2017
4,5,2018
5,6,2019
6,7,2020


In [82]:
df = df.merge(df_index_jurisdictions, on='jurisdiction', how='right').merge(df_index_years, on='year', how='right')

In [61]:
df.to_pickle("df_historical_deaths.pkl")
df_index_jurisdictions.to_pickle("df_index_jurisdictions_historical_deaths.pkl")
df_index_years.to_pickle("df_index_years.pkl")

In [87]:
df_historical = df.loc[:,['idx_jurisdiction','idx_year','week','number_of_deaths']]\
    .sort_values(['idx_jurisdiction','idx_year','week'])\
    .pivot_table(index=['idx_jurisdiction','idx_year'], values=['number_of_deaths'], columns=['week'], fill_value=0).astype(int)
df_historical

number_of_deaths                                \
week                                    1     2     3     4     5     6    
idx_jurisdiction idx_year                                                  
1                1                     355   872  1044  1022  1040   992   
                 2                    1139  1140  1043  1076  1110  1070   
                 3                    1008  1044  1029  1020  1103  1098   
                 4                    1121  1130  1048  1026  1036  1058   
                 5                    1248  1301  1275  1286  1147  1181   
...                                    ...   ...   ...   ...   ...   ...   
53               3                    1033  1042  1018  1054  1041  1013   
                 4                    1084  1092  1096  1111  1098  1126   
                 5                    1227  1164  1255  1153  1111  1176   
                 6                    1115  1056  1088  1103  1144  1033   
                 7                    1118  1060  1133  1086  1176  1055   

                                                   ...                    \
week                         7     8     9     10  ...    44    45    46   
idx_jurisdiction idx_year                          ...                     
1                1          942   990   958   988  ...   918   817   805   
                 2         1066  1102  1206  1103  ...   866   928   995   
                 3         1114   962  1006  1035  ...   952  1029  1004   
                 4         1060  1099  1081  1011  ...   994   994   982   
                 5         1096  1099   985  1059  ...   927   963  1055   
...                         ...   ...   ...   ...  ...   ...   ...   ...   
53               3         1094  1015  1068  1067  ...   948  1071   995   
                 4         1071  1024  1029  1037  ...  1026  1070  1047   
                 5         1099  1056  1064  1043  ...  1026   975  1029   
                 6         1078  1014  1031  1089  ...  1065  1079  1043   
                 7         1125  1161     0     0  ...     0     0     0   

                                                                     
week                         47    48    49    50    51    52    53  
idx_jurisdiction idx_year                                            
1                1          933   986   995  1036  1117  1068  1103  
                 2          974   914   899   768   778   912     0  
                 3         1029  1033  1057  1049  1134  1104     0  
                 4         1004  1067  1074  1120  1098  1080     0  
                 5         1009  1050  1044  1053  1105  1084     0  
...                         ...   ...   ...   ...   ...   ...   ...  
53               3          968   891  1023  1030  1079  1120     0  
                 4         1067   968  1034  1116  1071  1087     0  
                 5         1048  1021  1027  1083  1003  1012     0  
                 6         1074  1040  1090  1111  1056  1091     0  
                 7            0     0     0     0     0     0     0  

[371 rows x 53 columns]

In [99]:
stan_code = """
data {
    int<lower = 1> J; // number of jurisdictions
    int<lower = 1> Y; // number of calendar years
    int<lower = 1> W; // max week number
    int<lower = 1> Wmax[J, Y]; 

    int<lower = -1> deaths[J, Y, W]; // deaths counts
}

transformed data {
    vector[W] Theta;
    for (w in 1:W)
        Theta[w] = 2 * pi() * w / 52.1775;
}

parameters {
    vector[J] beta0;
    vector[J] beta1;
    vector[J] beta2;
    vector[J] beta3;
    vector[J] beta4;
    
    vector[Y] gamma[J];
}

model {
    /* priors */
    beta0 ~ std_normal();
    beta1 ~ std_normal();
    beta2 ~ std_normal();
    beta3 ~ std_normal();
    beta4 ~ std_normal();
    for (j in 1:J) 
        gamma[j] ~ std_normal();

    vector[W] lambda;
    int deaths_ji[W];
    for (j in 1:J) 
        for (i in 1:Y) {
            deaths_ji = to_array_1d(deaths[j, i, 1:W]);
            lambda = beta0[j] + beta1[j] * sin(Theta) + beta2[j] * cos(Theta) + beta3[j] * sin(Theta / 2.0) + beta4[j] * cos(Theta / 2.0) + gamma[j, i];
            target += poisson_log_lupmf(deaths_ji[1:Wmax[j, i]] | lambda[1:Wmax[j, i]]);
        }
}

generated quantities {
    int deaths_baseline[J, W];
    
    {
        vector[W] lambda;
        for (j in 1:J) {
            lambda = beta0[j] + beta1[j] * sin(Theta) + beta2[j] * cos(Theta) + beta3[j] * sin(Theta / 2.0) + beta4[j] * cos(Theta / 2.0) + gamma[j, Y];
            for (w in 1:W) 
                deaths_baseline[j, w] = poisson_log_rng(lambda[w]);
        }
    }
}
"""

In [103]:
## bash file
def bash_file(stanscriptdir):
    return """#!/bin/bash
cwd=$(pwd)
cd """+standistribdir+"""
make -j4 """+stanscriptdir+"""/fit
cd """+stanscriptdir+"""
mkdir -p diagnostics
for i in {1..8}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=1250 num_warmup=2000 thin=1 save_warmup=0 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished haha!
"""

In [104]:
def compile_model(filename_comment):
    # idxs_jurisdictions should start from 1 and be a range to the maximal value
    standirname = stanworkdir+'/baseline_'+filename_comment
    !mkdir -p {standirname}
    
    stanscriptdir = '../Dropbox/'+standirname[9:]
    
    jurisdictions = df_index_jurisdictions.jurisdiction.values
    years = df_index_years.year.values
    weeks = df_historical.columns.get_level_values('week').values
    
    df_index_jurisdictions.to_pickle(standirname+"/idx_jurisdictions.pkl")
    df_index_years.to_pickle(standirname+"/idx_years.pkl")
    
    df_historical.to_pickle(standirname+"/df_historical_death_counts.pkl")
    
    Deaths = np.zeros((len(df_historical.index.get_level_values('idx_jurisdiction').unique()),
                       len(df_historical.index.get_level_values('idx_year').unique()),
                       len(df_historical.columns.get_level_values('week').unique())), dtype='int')

    for j in range(len(jurisdictions)):
        Deaths[j, :, :] = df_historical.loc[lambda d: d.index.get_level_values('idx_jurisdiction')==j+1]

    # maximal non-zero week for given jurisdiction and snapshot
    ## +1 because the index count of an array starts from 1 in Stan, but from 0 in Python
    weekmax = np.matrix([[np.max(np.nonzero(x)) + 1 for x in Deaths[j, :, :]] for j in range(len(df_historical.index.get_level_values('idx_jurisdiction').unique()))])

    # data file
    stan_data = dict({
        'W': len(weeks),
        'Y': len(years),
        'J': len(jurisdictions),
        'Wmax': weekmax,
        # observed death counts
        'deaths': Deaths
    })
    pystan.misc.stan_rdump(stan_data, standirname+'/Data.R')

    # initial values
    stan_init = dict({
        'beta0': np.ones((stan_data['J']), dtype='float'),
        'beta1': np.ones((stan_data['J']), dtype='float'),
        'beta2': np.ones((stan_data['J']), dtype='float'),
        'beta3': np.ones((stan_data['J']), dtype='float'),
        'beta4': np.ones((stan_data['J']), dtype='float'),
        'gamma': np.ones((stan_data['J'], stan_data['Y']), dtype='float')
    })
    pystan.misc.stan_rdump(stan_init, standirname+'/Init.R')
    stan_init;

    # stan code
    f = open(standirname+"/fit.stan", "w")
    f.write(stan_code)
    f.close()

    # bash file
    f = open(standirname+"/fit_bash.sh", "w")
    f.write(bash_file(stanscriptdir))
    f.close()

    # compilation
    !rm -f {standirname+"/fit"}
    os.system("bash "+standirname+"/fit_bash.sh")
    
    return True

In [105]:
compile_model('final')

True

In [106]:
%%time
folder = 'baseline_final'
print(colored(folder, 'red'))
posterior_glob = !cd "{stanworkdir}/{folder}"; ls trace-*
cmdstan_data = az.from_cmdstan(posterior = [stanworkdir+"/"+folder+"/"+x for x in posterior_glob])

baseline_final
CPU times: user 8.01 s, sys: 142 ms, total: 8.15 s
Wall time: 9.99 s


In [108]:
%%time
df_output0 = get_stats_2d(cmdstan_data, ['deaths_baseline'])
df_output0[:11]

CPU times: user 52.5 s, sys: 0 ns, total: 52.5 s
Wall time: 52.5 s


,var,time,mean,hdi2.5,hdi25,hdi75,hdi97.5,q2.5,q25,median,q75,q97.5,ess_bulk,ess_tail,r_hat
0,deaths_baseline,"0,0",1094.522,1018.0,1067.0,1113.0,1159.0,1024.975,1071.0,1094.0,1117.0,1166.0,9402.0,10011.0,1.0
1,deaths_baseline,"0,1",1091.296,1016.0,1066.0,1113.0,1155.0,1022.000,1067.0,1091.0,1115.0,1161.0,10071.0,9396.0,1.0
2,deaths_baseline,"0,2",1087.046,1014.0,1062.0,1109.0,1153.0,1018.000,1063.0,1087.0,1111.0,1158.0,9757.0,9858.0,1.0
3,deaths_baseline,"0,3",1083.075,1015.0,1060.0,1106.0,1151.0,1015.000,1060.0,1083.0,1107.0,1152.0,10128.0,9898.0,1.0
4,deaths_baseline,"0,4",1077.774,1009.0,1049.0,1096.0,1143.0,1011.000,1054.0,1077.5,1101.0,1145.0,10058.0,9323.0,1.0
5,deaths_baseline,"0,5",1073.302,1007.0,1043.0,1089.0,1141.0,1007.000,1050.0,1072.0,1096.0,1142.0,9878.0,9896.0,1.0
6,deaths_baseline,"0,6",1068.095,999.0,1039.0,1086.0,1136.0,1000.000,1044.0,1068.0,1092.0,1138.0,9644.0,9615.0,1.0
7,deaths_baseline,"0,7",1062.462,993.0,1036.0,1082.0,1128.0,996.000,1039.0,1062.0,1086.0,1131.0,9424.0,9056.0,1.0
8,deaths_baseline,"0,8",1055.916,987.0,1030.0,1076.0,1124.0,986.000,1033.0,1056.0,1079.0,1124.0,9531.0,9463.0,1.0
9,deaths_baseline,"0,9",1048.939,980.0,1019.0,1065.0,1114.0,982.000,1026.0,1049.0,1072.0,1116.0,8872.0,9058.0,1.0
